In [1]:
# If you want to specify the package address
# you can add them to the PYTHONPATH environment variable.
# Also you can add them on the run time uncommenting the lines below
# import sys
# open3SPN2_HOME = '/Users/weilu/open3spn2/'
# openAWSEM_HOME = '/Users/weilu/openmmawsem/'
# sys.path.insert(0,open3SPN2_HOME)
# sys.path.insert(0,openAWSEM_HOME)

In [2]:
#Import openAWSEM, open3SPN2 and other libraries
import open3SPN2
import ffAWSEM

import pandas
import numpy as np
import simtk.openmm

from functools import partial
import sys

In [3]:
#Fix the system (adds missing atoms)
fix=open3SPN2.fixPDB("1lmb.pdb")

In [4]:
#Create a table containing both the proteins and the DNA
complex_table=open3SPN2.pdb2table(fix)

# Create a single memory file
ffAWSEM.create_single_memory(fix)

chain A is a DNA chain. it will be removed
chain B is a DNA chain. it will be removed
C 87
D 92


/home/cab22/anaconda3/envs/open3SPN2dev/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 4071
  warnings.warn(


In [5]:
#Generate a coarse-grained model of the DNA molecules
dna_atoms=open3SPN2.DNA.CoarseGrain(complex_table)

#Generate a coarse-grained model of the Protein molecules
protein_atoms=ffAWSEM.Protein.CoarseGrain(complex_table)

In [6]:
#Merge the models
Coarse=pandas.concat([protein_atoms,dna_atoms],sort=False)
Coarse.index=range(len(Coarse))
Coarse['serial']=list(Coarse.index)

In [7]:
#Save the protein_sequence
ffAWSEM.save_protein_sequence(Coarse,sequence_file='protein.seq')

In [8]:
# Create a merged PDB
ffAWSEM.writePDB(Coarse,'clean.pdb')

In [9]:
#Create the merged system

pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(ffAWSEM.xml,open3SPN2.xml)
s=forcefield.createSystem(top)

In [10]:
#Create the DNA and Protein Objects
dna=open3SPN2.DNA.fromCoarsePDB('clean.pdb')
dna.computeTopology(template_from_X3DNA=True)
with open('protein.seq') as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence_one)
dna.periodic=False
protein.periodic=False

In [11]:
#Copy the AWSEM parameter files
ffAWSEM.copy_parameter_files()

In [12]:
#Clear Forces from the system
keepCMMotionRemover=True
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'


In [13]:
#Initialize the force dictionary    
forces={}
for i in range(s.getNumForces()):
    force = s.getForce(i)
    force_name="CMMotionRemover"

#Add 3SPN2 forces
for force_name in open3SPN2.forces:
    print(force_name)
    force = open3SPN2.forces[force_name](dna)
    if force_name in ['BasePair','CrossStacking']:
        force.addForce(s)
    else:
        s.addForce(force)
    forces.update({force_name:force})

#Add AWSEM forces
openAWSEMforces = dict(Connectivity=ffAWSEM.functionTerms.basicTerms.con_term,
                       Chain=ffAWSEM.functionTerms.basicTerms.chain_term,
                       Chi=ffAWSEM.functionTerms.basicTerms.chi_term,
                       Excl=ffAWSEM.functionTerms.basicTerms.excl_term_v2,
                       rama=ffAWSEM.functionTerms.basicTerms.rama_term,
                       rama_pro=ffAWSEM.functionTerms.basicTerms.rama_proline_term,
                       contact=ffAWSEM.functionTerms.contactTerms.contact_term,
                       frag  = partial(ffAWSEM.functionTerms.templateTerms.fragment_memory_term, 
                                       frag_file_list_file="./single_frags.mem", 
                                       npy_frag_table="./single_frags.npy", 
                                       UseSavedFragTable=False, 
                                       k_fm=0.04184/3),
                       beta1 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_1,
                       beta2 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_2,
                       beta3 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_3,
                       pap1 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_1,
                       pap2 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_2,
                      )
protein.setup_virtual_sites(s)

#Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print(force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna,protein)
    s.addForce(force)
    forces.update({force_name: force})

#Fix exclussions
for force_name in openAWSEMforces:
    print(force_name)
    if force_name in ['contact']:
        force = openAWSEMforces[force_name](protein, withExclusion=True,periodic=False)
        print(force.getNumExclusions())
        #open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    elif force_name in ['Excl']:
        force = openAWSEMforces[force_name](protein)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
        #continue
    else:
        force = openAWSEMforces[force_name](protein)
    s.addForce(force)
    forces.update({force_name: force})

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
ExclusionProteinDNA
ElectrostaticsProteinDNA
Connectivity
Chain
Chi
Excl
0
639
rama
rama_pro
contact
Number of atom:  1171 Number of residue:  179
Contact cutoff  1.0 nm
NonbondedMethod:  1
670096
670096
frag
Loading Fragment files(Gro files)
Saving fragment table as npy file to speed up future calculation.
All gro files information have been stored in the ./single_frags.npy.             
You might want to set the 'UseSavedFragTable'=True to speed up the loading next time.             
But be sure to remove the .npy file if you modify the .mem file. otherwise it will keep using the old frag memeory.
beta1
beta_1 term ON
beta2
beta_2 term ON


/home/cab22/anaconda3/envs/open3SPN2dev/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


beta3
beta_3 term ON
pap1
pap_1 term ON
No ssweight given, assume all zero
pap2
pap_2 term ON
No ssweight given, assume all zero


In [14]:
# Set up the simulation
temperature=300 * simtk.openmm.unit.kelvin
platform_name='CPU' #'Reference','CPU','CUDA', 'OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-1175.1664712728998


In [15]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -1175.166472 kJ/mol
Bond 74.775959 kJ/mol
Angle 955.948472 kJ/mol
Stacking 203.567443 kJ/mol
Dihedral -390.606305 kJ/mol
BasePair -284.232239 kJ/mol
CrossStacking -47.586156 kJ/mol
Exclusion 23.99156 kJ/mol
Electrostatics 23.268273 kJ/mol
ExclusionProteinDNA 296.033548 kJ/mol
ElectrostaticsProteinDNA -10.459808 kJ/mol
Connectivity 1899.298172 kJ/mol
Chain 1899.298172 kJ/mol
Chi 1899.298172 kJ/mol
Excl 1899.298172 kJ/mol
rama -1363.522589 kJ/mol
rama_pro -1363.522589 kJ/mol
contact -1041.547712 kJ/mol
frag -1213.298352 kJ/mol
beta1 -300.79674 kJ/mol
beta2 -300.79674 kJ/mol
beta3 -300.79674 kJ/mol
pap1 -0.0 kJ/mol
pap2 -0.0 kJ/mol


In [16]:
#Add simulation reporters
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 100)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 100, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [17]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(1000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
100,0.20000000000000015,-4258.066062009481,163.6090344518113
200,0.4000000000000003,-4281.310173879332,212.45590263019517
300,0.6000000000000004,-4051.788518356684,223.2317554174403
400,0.8000000000000006,-3887.6592621312525,239.12095717626514
500,1.0000000000000007,-3865.353036820357,254.15091020550213
600,1.2000000000000008,-3721.8066221062236,252.1347424683077
700,1.400000000000001,-3769.7908515537674,270.0368792957282
800,1.6000000000000012,-3637.1104186820858,279.71085968781546
900,1.8000000000000014,-3594.506567345247,276.5867900632899
1000,2.0000000000000013,-3604.3429273586794,291.8782855428124


In [18]:
#Get the detailed energy after the simulation
energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

Bond 49.793502 kJ/mol
Angle 152.622548 kJ/mol
Stacking -410.943295 kJ/mol
Dihedral -461.214464 kJ/mol
BasePair -255.078782 kJ/mol
CrossStacking -51.577745 kJ/mol
Exclusion 0.699193 kJ/mol
Electrostatics 23.458813 kJ/mol
ExclusionProteinDNA -23.058345 kJ/mol
ElectrostaticsProteinDNA -11.928655 kJ/mol
Connectivity 1416.36526 kJ/mol
Chain 1416.36526 kJ/mol
Chi 1416.36526 kJ/mol
Excl 1416.36526 kJ/mol
rama -1510.099121 kJ/mol
rama_pro -1510.099121 kJ/mol
contact -1301.331233 kJ/mol
frag -1020.897119 kJ/mol
beta1 -201.153483 kJ/mol
beta2 -201.153483 kJ/mol
beta3 -201.153483 kJ/mol
pap1 -0.0 kJ/mol
pap2 -0.0 kJ/mol
